## 调整文件目录

In [1]:
""" 由于笔记本被收纳到单独的文件夹里，运行时 需且仅需一次 运行本模块来调整工作目录到项目目录下 """
import os


# 获取当前工作目录
current_dir = os.getcwd()
print("Current directory:", current_dir)

# 设置工作目录为上一级目录
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)
print("New working directory:", os.getcwd())

Current directory: e:\VSCodeFiles\quant\best\notebooks
New working directory: e:\VSCodeFiles\quant\best


## 正文

### 获取数据与数据预处理

In [2]:
import qstock as qs
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from WindPy import w
from Modules import Helper

import tushare as ts


token = '519550beb1e16be8a2bbd60fd5148d3d851074e6f5ec866832560432'
ts.set_token(token)
pro = ts.pro_api()
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

### barra


In [6]:
index_data = pd.read_csv('dataset\index_data\沪深300指数.csv')

In [128]:
mv_ret = index_data.pivot(columns='ts_code', index='trade_date', values='pct_chg')
mv_ret.index = mv_ret.index.astype(str)

stock_data = w.wsd('600000.SH', "close,open,high,low,pre_close,chg,pct_chg,volume,amt", Helper.convert_date_format_1(mv_ret.index[0]), Helper.convert_date_format_1(mv_ret.index[-1]), "")
stock_df = pd.DataFrame(np.array(stock_data.Data).T, columns=stock_data.Fields, index=stock_data.Times)
stock_ret = pd.DataFrame(stock_df['PCT_CHG'])

In [133]:
mv_ret.shape, stock_ret.shape

((4774, 1), (4774, 1))

In [134]:
class BarraBetaFactor:
    def __init__(self, window_size=252, half_life=63) -> None:
        super().__init__()
        self.window_size = window_size
        self.half_life = half_life
    
    @staticmethod
    def ewma_weights(length=252, half_life=63):
        """
        Calculate EWMA weights for a given length apnd half-life.

        Parameters:
        length (int): The number of data points.
        half_life (float): The half-life for the weights.

        Returns:
        np.ndarray: Array of EWMA weights.
        """
        # Calculate the smoothing factor alpha from the half-life
        alpha = 1 - np.exp(-np.log(2) / half_life)

        # Generate the weights
        weights = alpha * (1 - alpha) ** np.arange(length)[::-1]

        # Normalize the weights so they sum to 1
        weights /= weights.sum()

        return weights
    
    @staticmethod
    def get_beta_hsigma_window(X, Y, W):
        beta = np.linalg.pinv(X.T @ W @ X) @ X.T @ W @ Y
        hist_sigma = np.std(Y - X @ beta, ddof=0)
        return beta[1][0], hist_sigma
    
    def get_all(self, X_df, Y_df):
        _weights = np.diag(self.ewma_weights(self.window_size, self.half_life))
        if len(X_df) < self.window_size:
            return None
        
        _beta_list = []
        _hsigma_list = []
        for _idx in range(self.window_size, len(X_df) + 1):
            _X = np.c_[np.ones((self.window_size, 1)), X_df.values[(_idx - self.window_size): _idx, :]]
            _Y = Y_df.values[_idx - self.window_size: _idx, :]
            _beta, _sigma = self.get_beta_hsigma_window(_X, _Y, _weights)
            _beta_list.append(_beta)
            _hsigma_list.append(_sigma)

        return _beta_list, _hsigma_list
    

beta, hist_sigma = BarraBetaFactor().get_all(stock_ret, mv_ret)

In [142]:
import numpy as np
import pandas as pd

class BarraBetaFactor:
    def __init__(self, window_size=252, half_life=63) -> None:
        super().__init__()
        self.window_size = window_size
        self.half_life = half_life

    @staticmethod
    def ewma_weights(length=252, half_life=63):
        alpha = 1 - np.exp(-np.log(2) / half_life)
        weights = alpha * (1 - alpha) ** np.arange(length)[::-1]
        weights /= weights.sum()
        return weights

    @staticmethod
    def get_beta_hsigma_window(X, Y, W):
        beta = np.linalg.pinv(X.T @ W @ X) @ X.T @ W @ Y
        hist_sigma = np.std(Y - X @ beta, ddof=0)
        return beta[1][0], hist_sigma

    def get_all(self, X_df, Y_df):
        _weights = np.diag(self.ewma_weights(self.window_size, self.half_life))
        if len(X_df) < self.window_size:
            return None

        _beta_list = []
        _hsigma_list = []
        for _idx in range(self.window_size, len(X_df) + 1):
            _X = np.c_[np.ones((self.window_size, 1)), X_df.values[(_idx - self.window_size): _idx, :]]
            _Y = Y_df.values[_idx - self.window_size: _idx, :]
            _beta, _sigma = self.get_beta_hsigma_window(_X, _Y, _weights)
            _beta_list.append(_beta)
            _hsigma_list.append(_sigma)

        result_df = pd.DataFrame(index=X_df.index[self.window_size - 1:], columns=['beta', 'hist_sigma'])
        result_df['beta'] = _beta_list
        result_df['hist_sigma'] = _hsigma_list

        return result_df
    

res = BarraBetaFactor().get_all(stock_ret, mv_ret)
